# mmultの単体テスト

In [1]:
import numpy as np
import cffi

from pynq import Overlay

# load Base Overlay
Overlay("/home/xilinx/pynq/bitstream/base.bit").download()

import sys
sys.path.append("..")

from pynq_chainer import functions as F_
from pynq_chainer import links as L_
from pynq_chainer import overlays
from pynq_chainer import utils

pcsim is not run on pynq


In [22]:
from pynq.drivers import xlnk
memman = xlnk.xlnk()
memman.xlnk_reset()

In [3]:
mmult = overlays.Mmult()
ffi = cffi.FFI()

In [14]:
def debug_cdata(name, cdata):
    print(name)
    for i in range(5):
        print(cdata[i])

In [32]:
def test(debug=False):
    ffi = cffi.FFI()
    
    w_size = (1024, 32)
    x_size = (512, 32)
    w_size = (1, 784)
    x_size = (32, 784)
    w_size = (4, 4)
    x_size = (4, 4)

    w = np.random.uniform(-1, 1, w_size).astype(np.float32)
    x = np.random.uniform(-1, 1, x_size).astype(np.float32)

    x_nrows, x_ncols = x.shape
    w_nrows, w_ncols = w.shape
    #y = np.zeros((w_nrows, x_nrows)).astype(np.float32)
    y, y_cdata = utils.malloc_cma_ndarray((w_nrows, x_nrows))
    
    x, x_cdata = utils.copy_cma_ndarray(x)
    w, w_cdata = utils.copy_cma_ndarray(w)

    #x_cdata = ffi.from_buffer(x.data)
    #w_cdata = ffi.from_buffer(w.data)
    #y_cdata = ffi.from_buffer(y.data)
    
    print('x', x_cdata)
    print('w', w_cdata)
    print('y', y_cdata)
    debug_cdata("x", x_cdata)
    debug_cdata("w", w_cdata)
    debug_cdata("y", y_cdata)

    print('call')
    mmult(x_cdata, w_cdata, y_cdata, x_nrows, w_nrows, x_ncols)
    print('done')
    debug_cdata("y", y_cdata)
    y_ = x.dot(w.T)
    y = y.T
    
    debug_cdata("y", y_cdata)

    if debug:
        print("Actual(C):")
        print(y)
        print("Expected(NumPy):")
        print(y_)
    
    if np.allclose(y, y_, rtol=1e-04, atol=1e-04):
        print("OK")
    else:
        print("NG")

In [34]:
for i in range(1):
    test(debug=True)

cma alloc
cma alloc
cma alloc
x <cdata 'float *' 0x30908000>
w <cdata 'float *' 0x30907000>
y <cdata 'float *' 0x30909000>
x
0.3781375586986542
-0.45675113797187805
-0.6550893783569336
-0.7292472124099731
-0.9789353013038635
w
0.6785516738891602
0.3108958899974823
-0.49067652225494385
0.4745200276374817
0.3293452858924866
y
0.0
0.0
0.0
0.0
0.0
call
done
y
0.0
0.0
0.0
0.0
-0.3743168115615845
y
0.08997839689254761
-1.0227261781692505
-0.6401324272155762
0.14225299656391144
-0.3743168115615845
Actual(C):
[[ 0.0899784  -0.37431681 -0.33607054 -1.41942143]
 [-1.02272618  0.80610824 -0.50525182  1.88180518]
 [-0.64013243 -0.78106719 -0.05132452 -0.74017799]
 [ 0.142253   -0.06361359 -0.67438722 -1.11192119]]
Expected(NumPy):
[[ 0.0899784  -0.37431681 -0.33607054 -1.41942143]
 [-1.02272618  0.80610824 -0.50525182  1.88180518]
 [-0.64013243 -0.78106719 -0.05132452 -0.74017799]
 [ 0.142253   -0.06361359 -0.67438722 -1.11192119]]
OK


In [ ]:
memman.

In [ ]:
c